In [191]:
import torch
import torch.nn as nn
device = torch.device('cuda:0')
import mexhat_ops
import gemm_ops
import numpy as np
from tqdm import tqdm
import time

def gtfwd(a, b, c, d):
    return mexhat_ops.forward(a, b.T.contiguous(), c.T.contiguous(), d.T.contiguous(), )

In [193]:
batch_size, in_feats, out_feats, n2 = 128, 768, 512, 512


I = 200
for i in tqdm(range(I)):
    x = torch.randn(batch_size, in_feats).to(device)
    scale1 = torch.randn(in_feats, out_feats).to(device)
    bias1 = torch.randn(in_feats, out_feats).to(device)
    weight1 = torch.randn(in_feats, out_feats).to(device)

    scale2 = torch.randn(out_feats, n2).to(device)
    bias2 = torch.randn(out_feats, n2).to(device)
    weight2 = torch.randn(out_feats, n2).to(device)


    gt = gtfwd(x, scale1, bias1, weight1)
    y = gemm_ops.forward(x, scale1, bias1, weight1)

    time.sleep(0.5)

    # gt = gtfwd(gt, scale2, bias2, weight2)
    # y = gemm_ops.forward(y, scale2, bias2, weight2)

    reltol = 1e-5
    reldiff = reltol * gt
    print(f'max diff = {abs(gt-y).max()}')
    diff = abs(gt-y).mean() - abs(reldiff).mean()
    ratio = abs(gt-y).mean() / abs(reldiff).mean()

    if diff > 0:
        rd = ratio * reltol
        print(rd.item())
        print('\n' + '-'*10 + 'fail' + '-'*10)
        print(gt)
        print(y)
        break

    if i % 100 == 0:
        z = ratio * reltol
        print(z)

    if i == I - 1:
        print('correct')



  0%|          | 1/200 [00:00<02:12,  1.50it/s]

max diff = 6.4849853515625e-05
tensor(4.4799e-07, device='cuda:0')


  1%|          | 2/200 [00:01<02:08,  1.53it/s]

max diff = 6.4849853515625e-05


  2%|▏         | 3/200 [00:01<02:07,  1.55it/s]

...

  6%|▌         | 12/200 [00:07<01:52,  1.66it/s]

max diff = 8.392333984375e-05


  6%|▋         | 13/200 [00:08<01:55,  1.62it/s]

max diff = 7.2479248046875e-05


KeyboardInterrupt: 

In [23]:
import torch
import torch.nn as nn
device = torch.device('cuda:0')
import mmops
import time
import numpy as np

In [190]:
M, N, K = 128, 128, 128
B = 256

I = 1
for i in range(I):
    a = torch.randn(M, K).to(device)
    b = torch.randn(K, N).to(device)
    c = torch.randn(N, B).to(device)

    u = nn.functional.linear(a, b.T)
    w = mmops.matmul_new(a, b)

    u = nn.functional.linear(u, c.T)
    w = mmops.matmul(w, c)

    reltol = 1e-5
    reldiff = reltol * u
    print(abs(u-w).max())
    print(abs(u-w).mean())
    # diff = abs(u-w).mean() - abs(reldiff).mean()
    # ratio = abs(u-w).mean() / abs(reldiff).mean()

    # if diff > 0:
    #     print(diff)
    #     print(u)
    #     print(w)
    #     print('fail')
    #     break
    # if i == I - 1:
    #     print('correct')
    


tensor(0.0002, device='cuda:0')
tensor(3.0772e-05, device='cuda:0')


In [143]:
print((u-w).mean())

tensor(-1.1721e-09, device='cuda:0')


In [114]:
b

tensor([[-1.1335,  0.7884,  2.3621,  ..., -0.9079, -1.6368, -1.5175],
        [-2.3480,  1.0714, -0.9907,  ..., -2.0680, -0.6640, -0.9950],
        [-0.6249, -0.3968,  0.7080,  ...,  0.1292,  0.4197,  0.2534],
        ...,
        [-0.8842,  1.7174,  1.2561,  ..., -1.4580, -0.5914,  1.2905],
        [-1.3811,  1.3207,  0.3373,  ...,  1.0310,  0.6368, -1.4615],
        [ 0.0826,  1.3893, -0.4246,  ...,  0.3244,  1.0722,  0.7935]],
       device='cuda:0')

In [115]:
a@b

tensor([[ 19.6997,  -9.1748,  24.6209,  ...,  -9.3182,  -8.7224,   2.2500],
        [  5.8683,  -1.7790, -17.8242,  ...,  -1.5462,  -6.6428,  -0.8986],
        [-30.3215,  15.0222,  -0.3260,  ...,  11.7170,  -3.5433,   4.2187],
        ...,
        [ 15.7976, -15.8519,   7.6004,  ..., -12.3417, -15.5606,  -1.8131],
        [  8.5211,  -0.2519,  21.4157,  ...,   1.9600, -18.2365,   4.2892],
        [  0.4466,  -4.4348, -13.0536,  ...,   1.4966,   5.1205,  -2.6607]],
       device='cuda:0')

In [116]:
w


tensor([[ 19.6997,  -9.1748,  24.6209,  ...,  -9.3182,  -8.7224,   2.2500],
        [  5.8683,  -1.7790, -17.8242,  ...,  -1.5462,  -6.6428,  -0.8986],
        [-30.3215,  15.0222,  -0.3260,  ...,  11.7170,  -3.5433,   4.2187],
        ...,
        [ 15.7976, -15.8519,   7.6004,  ..., -12.3417, -15.5606,  -1.8131],
        [  8.5211,  -0.2519,  21.4157,  ...,   1.9600, -18.2365,   4.2892],
        [  0.4466,  -4.4348, -13.0536,  ...,   1.4966,   5.1205,  -2.6607]],
       device='cuda:0')

In [123]:
abs((u-w))

tensor([[1.4305e-06, 0.0000e+00, 2.3842e-06,  ..., 1.8775e-06, 2.3842e-07,
         2.3842e-07],
        [0.0000e+00, 7.1526e-07, 7.1526e-07,  ..., 8.3447e-07, 9.5367e-07,
         1.4305e-06],
        [2.8610e-06, 9.5367e-07, 0.0000e+00,  ..., 1.9073e-06, 7.1526e-07,
         2.8610e-06],
        ...,
        [4.2915e-06, 3.8147e-06, 0.0000e+00,  ..., 1.4305e-06, 4.7684e-07,
         1.6689e-06],
        [4.7684e-07, 5.7220e-06, 4.7684e-06,  ..., 0.0000e+00, 4.7684e-07,
         1.6689e-06],
        [9.5367e-07, 9.5367e-07, 0.0000e+00,  ..., 9.5367e-07, 9.5367e-07,
         2.6226e-06]], device='cuda:0')

In [124]:
abs(u)

tensor([[ 7.2985,  2.4518,  7.1181,  ...,  0.3825,  2.7120,  1.9256],
        [18.7307,  3.2470,  3.9668,  ...,  1.9174,  4.8413,  5.5221],
        [10.3170,  4.2650, 17.4981,  ..., 10.4715,  0.4542, 10.9597],
        ...,
        [ 7.9962, 20.9019, 11.6732,  ...,  5.3086,  2.5294,  2.2046],
        [ 5.3072, 11.3881, 10.6650,  ...,  4.6941,  5.5714,  1.4418],
        [ 9.2655, 10.0611,  4.9043,  ...,  4.8464,  6.6865,  3.8165]],
       device='cuda:0')